In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [6]:
df = pd.read_csv("data/Capital_Deliver_Project_deidentified.csv")
df.head()

,ID,FFC,Delivery Program,Tender Closed,Owner Group,BNI,BCA,PBC,EPT,Approval body,Owner Team,Stage1,Stage2,Stage3
0,Q05388,2359288.62,Major Works - Framework,NaN,Corporate Services,5/02/2016 5:00:00 PM,2016-05-25 17:00:00,30/03/2016 5:00:00 PM,3 Gate Major Infrastructure,People Leadership Group,General Counsel,2017-02-16 00:00:00,2017-12-22 17:00:00,2018-02-16 17:00:00
1,Q05640,2066075.59,Major Works - Framework,NaN,Corporate Services,11/07/2016 5:00:00 PM,2017-02-28 08:00:00,24/10/2016,2 Gate Major Infrastructure,People Leadership Group,General Counsel,2017-08-11 17:00:00,2017-12-07 17:00:00,2019-06-06 17:00:00
2,Q05641,1066164.85,Major Works - Framework,NaN,Corporate Services,24/06/2016 5:00:00 PM,2017-02-28 08:00:00,24/10/2016,2 Gate Major Infrastructure,People Leadership Group,General Counsel,2017-08-15 17:00:00,2017-12-21 17:00:00,2018-08-15 17:00:00
3,Q05643,1122486.96,Major Works - Framework,NaN,Corporate Services,11/07/2016,2017-02-28 10:00:00,24/10/2016,2 Gate Major Infrastructure,People Leadership Group,General Counsel,2017-08-15 17:00:00,2017-12-11 17:00:00,2018-08-15 17:00:00
4,Q05664,636484.75,Major Works - Framework,NaN,Corporate Services,11/07/2016 8:57:00 AM,2017-02-27 00:00:00,24/10/2016,2 Gate Major Infrastructure,People Leadership Group,General Counsel,2017-08-11 17:00:00,2017-12-07 17:00:00,2019-06-11 17:00:00


# Convert to dates

In [17]:
df["BNI"] = pd.to_datetime(df['BNI'])
df["BCA"] = pd.to_datetime(df['BCA'])
df["PBC"] = pd.to_datetime(df['PBC'])
df["Stage1"] = pd.to_datetime(df['Stage1'])
df["Stage2"] = pd.to_datetime(df['Stage2'])
df["Stage3"] = pd.to_datetime(df['Stage3'])

/var/folders/bl/615qdshd0qj1cddydmjsmf800000gn/T/ipykernel_52824/3234553480.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["PBC"] = pd.to_datetime(df['PBC'])


In [21]:
df.dtypes

ID                          object
FFC                        float64
Delivery Program            object
Tender Closed               object
Owner Group                 object
BNI                 datetime64[ns]
BCA                 datetime64[ns]
PBC                 datetime64[ns]
EPT                         object
Approval body               object
Owner Team                  object
Stage1              datetime64[ns]
Stage2              datetime64[ns]
Stage3              datetime64[ns]
dtype: object

# Normalise data using MinMax scaling

In [23]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

ValueError: could not convert string to float: 'Q05388'

# Sequence creation for training

In [22]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

SEQ_LENGTH = 5  # Number of time steps
X, y = create_sequences(scaled_values, SEQ_LENGTH)

# Split into training and test sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

NameError: name 'scaled_values' is not defined

# Building the LSTM 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(50, activation='relu', input_shape=(SEQ_LENGTH, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model

In [ ]:
BATCH_SIZE = 5  # or 8
SEQ_LENGTH = 3   # Try reducing this

X, y = create_sequences(scaled_values, SEQ_LENGTH)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Rebuild model with updated parameters
model = Sequential([
    LSTM(50, activation='relu', input_shape=(SEQ_LENGTH, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model with reduced batch size
history = model.fit(X_train, y_train, epochs=50, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))

# Testing

In [ ]:
import matplotlib.pyplot as plt

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# Make predictions
predictions = model.predict(X_test)

# Inverse transform to original scale
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(np.arange(len(y_test)), y_test, label='True Values')
plt.plot(np.arange(len(predictions)), predictions, label='Predictions')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Predictions and true values
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate metrics
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")